# 二进制文件读取与导出

本节主要介绍二进制文件（ Excel ）的读取与导出操作。

In [ ]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

## Excel 文件读写

Excel 是微软公司办公软件 Office 的重要组成部分，可以进行各种数据处理，统计分析与辅助决策操作。在企事业管理、财经和金融等众多领域得到广泛应用。Excel 的输出文件是二进制格式文件，包括两种：
-  Office 2007之前输出的文件，后缀为`.xls`；
-  Office 2007之后输出的文件，后缀为`.xlsx`。

本节提供有两个 excel 示例文件，一个是`xls`文件，另一个是`xlsx`文件，来自于 Pandas 测试用例：
```
../data/test_multisheet.xls
../data/test_multisheet.xlsx
```

两个 excel 文件内容一样，包含三个表格，表格包括4列内容，如下图所示：
![excel 文件](../images/pandas_excelfile.png)

### `pd.read_excel()`函数

Pandas 使用`pd.read_excel() ` 函数来读取 excel 文件的表格数据。在内部会使用 `xlrd` 与 `openpyxl` 库来分别读取`xls`与`xlsx`格式文件。使用Anaconda套件无需安装，在其它情况可能需要使用`pip`工具进行安装。

`pd.read_excel()`使用语法为：
```python
pd.read_excel(io, sheet_name=0, header=0, names=None, index_col=None, usecols=None, squeeze=False, dtype=None, engine=None, converters=None, true_values=None, false_values=None, skiprows=None, nrows=None, na_values=None, parse_dates=False, date_parser=None, thousands=None, comment=None, skipfooter=0, convert_float=True, **kwds)
```
主要参数包括：
- `io`，文件路径。
- `sheepname=0`，字符串或编号。Excel文件的表格的名字或位置；
- `header=0`，将某行做诶列名，默认是第一行。如果没有列名的话，设为 None。
- `names`，结果的列名列表，与`header=None`配合使用。
- `index_col`，用作行索引的列编号或列名。可以是名称/数字标量，也可以是分层索引。
- `dtype`，指定每列的数据类型。
- `skiprows`，从文件开头起，需要跳过的行数或行号列表。
- `na_values`，需要用 NA 替换的值序列
- `comment=None`，在行结尾处分隔注释的字符
- `parse_dates=False`，尝试将数据解析为日期。默认是`False`。如果为`True`，则尝试解析所有列。可以指定列号或列名列表对指定列进行解析。
- `date_parser`，用于解析日期的函数
- `nrows`，从文件开头处读入的行数
- `skip_footer`，忽略文件尾部的行数。
- `squeeze`，如果解析数据只包含一列，返回一个 `Series` 对象

### 读取示例

下面调用`pd.read_excel()`来读取指定表的数据：

In [ ]:
xlsfile = os.path.join('..', 'data', 'test_multisheet.xls')
xlsxfile = os.path.join('..', 'data', 'test_multisheet.xlsx')

In [ ]:
# 读取第一个表数据
df = pd.read_excel(xlsfile)
df.info()

In [ ]:
df.head()

可以看出，`date`列自动解析为日期数据，同时自动创建一个整数索引对象。下面使用参数`index_col`把`date`列设置为索引：

In [ ]:
# 读取 Alpha 表数据
df = pd.read_excel(xlsxfile, 'Alpha', index_col=0)
df.info()

In [ ]:
df.head()

## 导出 Excel 文件

使用`DataFrame`对象的`to_excel()`方法，可以把将数据存储为 Excel 文件，其使用语法为：
```python
df.to_excel(excel_writer, sheet_name='Sheet1', na_rep='', float_format=None, columns=None, header=True, index=True, index_label=None, startrow=0, startcol=0, engine=None, merge_cells=True, encoding=None, inf_rep='inf', verbose=True, freeze_panes=None)
```
主要参数
- `excel_writer`， excel 文件路径或 ExcelWriter 实例对象；
- `sheet_name='Sheet1'`，表格名称

下面把上面的`DataFrame`对象写入到 excel 文件中去：

In [ ]:
df.to_excel('output01.xlsx', 'NewCharlie')

## 更多二进制文件读写

除了 excel 格式文件外，Pandas支持更多二进制文件格式读写，例如：
- `pickle`，Python 内置 pickle 模块序列化的二进制数据格式
- `hdf`，HDF5 是一个用于存储大量的科学数据的文件格式
- `msgpack`，Pandas 内置支持的 MessagePack 格式
- `feather`，R 语言社区常用的跨语言列式文件格式
- `sas`，SAS 系统中的SAS数据集
- `stata`， Stata 二进制文件
- `parquet`，读取 Apache Parquet 二进制格式文件

### pickle 文件读写

Pandas 提供`pd.read_pickle()`来读取 pickle 二进制文件。`DataFrame`对象的`to_pickle()`方法可以将数据写为pickle格式。

下面语句首先读取 Excel 文件数据，然后存储为 pickle 格式文件：

In [ ]:
xlsxfile = os.path.join('..', 'data', 'test_multisheet.xlsx')
# 读取 Alpha 表数据
df = pd.read_excel(xlsxfile, 'Alpha')
# 输出 pickle 格式文件
df.to_pickle('output01.pkl')

In [ ]:
%ls *.pkl

下面使用`pd.read_pickle()`来读取 pickle 序列化文件：

In [ ]:
df = pd.read_pickle('output01.pkl')
df.info()

建议把 pickle 格式作为短期应用的存储格式。理由是 pickle 无法保证其格式的长期有效性，会由于版本更新导致无法发序列化一个 pickle 对象。

### HDF5 文件读写

HDF5 是一个用于存储大量的科学数据的文件格式，每个 HDF5 文件可以存储多个数据集并支持元数据。Pandas 提供了一个`HDFStore`类，可以像处理字典那样来简化数据集的存储。下面把 Excel 文件中读取的数据初存储到 HDF5 中。

In [ ]:
xlsxfile = os.path.join('..', 'data', 'test_multisheet.xlsx')
h5file = 'output.h5'
with pd.HDFStore(h5file) as store:
    for name in ['Charlie', 'Alpha', 'Beta']:
        # 读取 excel 表数据
        df = pd.read_excel(xlsxfile, name)
        store[name] = df

In [ ]:
%ls *.h5

可以使用`HDFStore`类来读取，也可以使用`pd.read_hdf()`函数来快速读取数据：

In [ ]:
df = pd.read_hdf(h5file, 'Charlie')
df.head()

### Parquet文件

Aparche Parquet 文件是专门为分布式存储而设计的二进制格式文件。如果数据分析涉及到分布式存储，可以尝试使用一下。